In [10]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 11: Hex Ed ---</h2><p>Crossing the bridge, you've barely reached the other side of the stream when a program comes up to you, clearly in distress.  "It's my child process," she says, "he's gotten lost in an infinite grid!"</p>
<p>Fortunately for her, you have plenty of experience with infinite grids.</p>
<p>Unfortunately for you, it's a <a href="https://en.wikipedia.org/wiki/Hexagonal_tiling">hex grid</a>.</p>
<p>The hexagons ("hexes") in <span title="Raindrops on roses and whiskers on kittens.">this grid</span> are aligned such that adjacent hexes can be found to the north, northeast, southeast, south, southwest, and northwest:</p>
<pre><code>  \ n  /
nw +--+ ne
  /    \
-+      +-
  \    /
sw +--+ se
  / s  \
</code></pre>
<p>You have the path the child process took. Starting where he started, you need to determine the fewest number of steps required to reach him. (A "step" means to move from the hex you are in to any adjacent hex.)</p>
<p>For example:</p>
<ul>
<li><code>ne,ne,ne</code> is <code>3</code> steps away.</li>
<li><code>ne,ne,sw,sw</code> is <code>0</code> steps away (back where you started).</li>
<li><code>ne,ne,s,s</code> is <code>2</code> steps away (<code>se,se</code>).</li>
<li><code>se,sw,se,sw,sw</code> is <code>3</code> steps away (<code>s,s,sw</code>).</li>
</ul>
</article>


In [11]:
from functools import cache
from math import ceil

from more_itertools import first


tests_partI = [
    {
        "name": "Example 1",
        "directions": "ne,ne,ne",
        "expected": 3,
    },
    {
        "name": "Example 2",
        "directions": "ne,ne,sw,sw",
        "expected": 0,
    },
    {
        "name": "Example 3",
        "directions": "ne,ne,s,s",
        "expected": 2,
    },
    {
        "name": "Example 4",
        "directions": "se,sw,se,sw,sw",
        "expected": 3,
    },
    {
        "name": "Example 5",
        "directions": "ne,nw,ne,nw",
        "expected": 2,
    },
    {
        "name": "Example 5",
        "directions": "ne,se,ne,se",
        "expected": 4,
    },
    {
        "name": "Example 5",
        "directions": "nw,sw,nw",
        "expected": 3,
    },
]


#   \    /    \    /    \    /    \    /    \    /    \    /    \    /
#    +--+      +--+      +--+      +--+      +--+      +--+      +--+
#   /    \    /    \    /    \    /    \    /    \    /    \    /    \
# -+      +--+      +--+      +--+      +--+      +--+      +--+      +-
#   \    /    \    /    \    /    \    /    \    /    \    /    \    /
#    +--+      +--+      +--+      +--+      +--+      +--+      +--+
#   /    \    /    \    /    \    /    \    /    \    /    \    /    \
# -+      +--+      +--+      +--+ 0, 0 +--+      +--+      +--+      +-
#   \    /    \    /    \    /    \    /    \    /    \    /    \    /
#    +--+      +--+      +--+      +--+      +--+      +--+      +--+
#   /    \    /    \    /    \    /    \    /    \    /    \    /    \
# -+      +--+      +--+      +--+      +--+      +--+      +--+      +-
#   \    /    \    /    \    /    \    /    \    /    \    /    \    /
#    +--+      +--+      +--+      +--+      +--+      +--+      +--+
#   /    \    /    \    /    \    /    \    /    \    /    \    /    \
# -+      +--+      +--+      +--+      +--+      +--+      +--+      +-
#   \    /    \    /    \    /    \    /    \    /    \    /    \    /
#    +--+      +--+      +--+      +--+      +--+      +--+      +--+
#   /    \    /    \    /    \    /    \    /    \    /    \    /    \
# -+      +--+      +--+      +--+      +--+      +--+      +--+      +-
#   \ n  /    \ n  /    \ n  /    \ n  /    \ n  /    \ n  /    \ n  /
# nw +--+ nenw +--+ nenw +--+ nenw +--+ nenw +--+ nenw +--+ nenw +--+ ne
#   /    \    /    \    /    \    /    \    /    \    /    \    /    \
# -+      +--+      +--+      +--+      +--+      +--+      +--+      +-
#   \    /    \    /    \    /    \    /    \    /    \    /    \    /
# sw +--+ sesw +--+ sesw +--+ sesw +--+ sesw +--+ sesw +--+ sesw +--+ se
#   / s  \    / s  \    / s  \    / s  \    / s  \    / s  \    / s  \
# -+      +--+      +--+      +--+      +--+      +--+      +--+      +-
#   \    /    \    /    \    /    \    /    \    /    \    /    \    /
#    +--+      +--+      +--+      +--+      +--+      +--+      +--+
def fewest_number_of_steps(directions: str) -> int:
    moves = {
        "n": (-1, 0),
        "ne": (-1 / 2, 1),
        "se": (1 / 2, 1),
        "s": (1, 0),
        "sw": (1 / 2, -1),
        "nw": (-1 / 2, -1),
    }
    max_steps = 0
    r, q = 0, 0
    for m in directions.split(","):
        dr, dq = moves[m]
        r += dr
        q += dq
        max_steps = max(max_steps, calc_steps(r, q))

    return calc_steps(r, q), max_steps


@cache
def calc_steps(r, q):
    r, q = abs(r), abs(q)

    steps = 0
    while r > 0 and q > 0:
        r -= 1 / 2
        q -= 1
        steps += 1

    return ceil(steps + r + q)


run_tests_params(
    lambda directions: first(fewest_number_of_steps(directions)), tests_partI
)


Test Example 1 passed, for <lambda>.
Test Example 2 passed, for <lambda>.
Test Example 3 passed, for <lambda>.
Test Example 4 passed, for <lambda>.
Test Example 5 passed, for <lambda>.
Test Example 5 passed, for <lambda>.
Test Example 5 passed, for <lambda>.
Success


In [12]:
with open("../input/day11.txt") as f:
    puzzle = f.read().strip()

partI, partII = fewest_number_of_steps(puzzle)

print(f"part I: {partI}")

part I: 808


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>808</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p><em>How many steps away</em> is the <em>furthest</em> he ever got from his starting position?</p>
</article>

</main>


In [13]:
print(f"part II: {partII}")

part II: 1556


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1556</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>


<link href="style.css" rel="stylesheet"></link>


<link href="style.css" rel="stylesheet"></link>
